In [1]:
#Importing needed libraries
import pandas as pd

In [44]:
#Download Medicaid Enrolled Provider Listing from
#https://health.data.ny.gov/Health/Medicaid-Enrolled-Provider-Listing/keti-qx5t

df = pd.read_csv('https://health.data.ny.gov/api/views/keti-qx5t/rows.csv?accessType=DOWNLOAD')

In [45]:
df.columns

Index(['MEDICAID PROVIDER ID', 'NPI', 'PROVIDER OR FACILITY NAME',
       'MEDICAID TYPE', 'PROFESSION OR SERVICE', 'PROVIDER SPECIALTY',
       'SERVICE ADDRESS', 'CITY', 'STATE', 'ZIP CODE', 'COUNTY', 'TELEPHONE',
       'LATITUDE', 'LONGITUDE', 'ENROLLMENT BEGIN DATE',
       'NEXT ANTICIPATED REVALIDATION DATE', 'FILE DATE'],
      dtype='object')

In [46]:
df.head(2)

,MEDICAID PROVIDER ID,NPI,PROVIDER OR FACILITY NAME,MEDICAID TYPE,PROFESSION OR SERVICE,PROVIDER SPECIALTY,SERVICE ADDRESS,CITY,STATE,ZIP CODE,COUNTY,TELEPHONE,LATITUDE,LONGITUDE,ENROLLMENT BEGIN DATE,NEXT ANTICIPATED REVALIDATION DATE,FILE DATE
0,13098,1.477673e+09,JCCA FH DIVISION,FFS,CHILD (FOSTER) CARE AGENCIES,NaN,1075 BROADWAY,PLEASANTVILLE,NY,10570-2345,WESTCHESTER,NaN,41.13006,-73.77775,02/01/1979,12/11/2020,04/20/2020
1,13098,1.477673e+09,JCCA FH DIVISION,FFS,CHILD (FOSTER) CARE AGENCIES,NaN,111 N 3RD AVE VERNONDALE RES,MOUNT VERNON,NY,10550-1364,WESTCHESTER,NaN,40.91446,-73.83636,02/01/1979,12/11/2020,04/20/2020


In [47]:
df_export = pd.DataFrame()

In [48]:
df_export['LATITUDE'] = df['LATITUDE']
df_export['LONGITUDE'] = df['LONGITUDE']
#df_export['ENROLLMENT BEGIN DATE'] = df['ENROLLMENT BEGIN DATE']

In [49]:
df_export.dropna(inplace=True)
df_export.drop_duplicates(inplace=True)

In [50]:
df_export.sort_values(by=['LATITUDE']).head(2)

,LATITUDE,LONGITUDE
674483,17.73421,-64.73469
544308,19.71612,-155.99726


In [51]:
#df_export.to_csv("df_export.csv")

### Download NTA shapefiles 
For this part it is needed to download the NTA shapefiles from the NYC Open Data

https://data.cityofnewyork.us/City-Government/Neighborhood-Tabulation-Areas-NTA-/cpf4-rkhq

And unzip the file in the same folder as this notebook.

In [52]:
import geopandas as gpd
import fiona
import fiona.crs
import shapely
import rtree

In [53]:
# This is to load the shape file
shapefile = 'Neighborhood Tabulation Areas (NTA)\geo_export_d577b763-d73a-455e-a44f-8deffcc79461.shp'

# And project it into EPSG:2263 (NAD 83 NY State) plane
neighborhoods = gpd.read_file(shapefile).to_crs(fiona.crs.from_epsg(2263))

In [54]:
neighborhoods.head(2)

,boro_code,boro_name,county_fip,ntacode,ntaname,shape_area,shape_leng,geometry
0,3.0,Brooklyn,047,BK88,Borough Park,5.400502e+07,39247.227831,"POLYGON ((990897.900 169268.121, 990588.252 16..."
1,4.0,Queens,081,QN51,Murray Hill,5.248828e+07,33266.904797,"POLYGON ((1038593.459 221913.355, 1039369.281 ..."


In [55]:
df_indexed = df_export.copy()
df_indexed.head(2)

,LATITUDE,LONGITUDE
0,41.13006,-73.77775
1,40.91446,-73.83636


In [78]:
index = rtree.Rtree()
for idx,geometry in enumerate(neighborhoods.geometry):
    index.insert(idx, geometry.bounds)

In [64]:
import csv
import pyproj
import shapely.geometry as geom
proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    

In [83]:
for dfindex, row in df_indexed.iterrows():
    p = geom.Point(proj(float(row['LONGITUDE']), float(row['LATITUDE'])))
    for idx in index.intersection((p.x, p.y, p.x, p.y)):
        if neighborhoods.geometry[idx].contains(p):
            df_indexed.loc[(df_indexed["LATITUDE"]==float(row['LATITUDE']))&(df_indexed["LONGITUDE"]==float(row['LONGITUDE'])),'NTA'] = neighborhoods.iloc[idx ,]['ntaname']

In [96]:
#BASE CODE FOR BIG DATA

#with open('df_export.csv', 'r') as fi:
#    reader = csv.reader(fi)
#    #next(reader)
#    for row in reader:
#        if len(row[0])==0 : continue
#        p = geom.Point(proj(float(row[2]), float(row[1])))
#        for idx in index.intersection((p.x, p.y, p.x, p.y)):
#            if neighborhoods.geometry[idx].contains(p):
#                df_indexed.loc[(df_indexed["LATITUDE"]==float(row[1]))&(df_indexed["LONGITUDE"]==float(row[2])),'NTA'] = neighborhoods.iloc[idx ,]['ntaname']

In [84]:
df_indexed

,LATITUDE,LONGITUDE,NTA
0,41.13006,-73.77775,NaN
1,40.91446,-73.83636,NaN
2,40.72684,-73.79740,Pomonok-Flushing Heights-Hillcrest
3,40.72397,-73.80616,Kew Gardens Hills
4,40.63005,-73.95919,Flatbush
...,...,...,...
684783,28.35825,-81.38299,NaN
684825,40.68087,-75.22013,NaN
684826,42.59787,-71.80637,NaN
684838,42.09517,-80.14449,NaN


In [85]:
df_indexed.dropna(inplace=True)

In [86]:
df_analysis = pd.DataFrame()

In [87]:
df_analysis['ENROLLMENT BEGIN DATE'] = df['ENROLLMENT BEGIN DATE']
df_analysis['LATITUDE'] = df['LATITUDE']
df_analysis['LONGITUDE'] = df['LONGITUDE']

In [88]:
df_analysis = df_analysis.merge(df_indexed, on=(['LATITUDE','LONGITUDE']))

In [89]:
df_analysis['YEAR'] = pd.DatetimeIndex(df_analysis['ENROLLMENT BEGIN DATE']).year

In [90]:
df_analysis = df_analysis[df_analysis['YEAR'] >= 2011]

In [91]:
df_analysis

,ENROLLMENT BEGIN DATE,LATITUDE,LONGITUDE,NTA,YEAR
15,10/24/2012,40.72684,-73.79740,Pomonok-Flushing Heights-Hillcrest,2012
16,10/10/2011,40.72684,-73.79740,Pomonok-Flushing Heights-Hillcrest,2011
17,08/03/2011,40.72684,-73.79740,Pomonok-Flushing Heights-Hillcrest,2011
18,08/03/2011,40.72684,-73.79740,Pomonok-Flushing Heights-Hillcrest,2011
19,03/28/2012,40.72684,-73.79740,Pomonok-Flushing Heights-Hillcrest,2012
...,...,...,...,...,...
267329,03/01/2020,40.85655,-73.84154,Pelham Parkway,2020
267330,09/10/2019,40.76715,-73.92144,Old Astoria,2019
267331,03/01/2020,40.74663,-73.98295,Midtown-Midtown South,2020
267332,09/14/2019,40.64080,-73.98284,Borough Park,2019


In [92]:
df_analysis_pivot = pd.pivot_table(df_analysis, index='NTA',values='ENROLLMENT BEGIN DATE',columns='YEAR', aggfunc=lambda x: len(x))

In [93]:
df_analysis_pivot

YEAR,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
NTA,,,,,,,,,,
Airport,NaN,3.0,NaN,3.0,3.0,NaN,NaN,NaN,1.0,1.0
Allerton-Pelham Gardens,15.0,24.0,56.0,51.0,30.0,41.0,77.0,63.0,29.0,NaN
Annadale-Huguenot-Prince's Bay-Eltingville,38.0,64.0,67.0,84.0,63.0,141.0,127.0,122.0,111.0,3.0
Arden Heights,NaN,2.0,24.0,8.0,4.0,3.0,5.0,4.0,1.0,NaN
Astoria,66.0,72.0,64.0,116.0,77.0,105.0,142.0,151.0,96.0,3.0
...,...,...,...,...,...,...,...,...,...,...
Yorkville,52.0,91.0,86.0,136.0,111.0,140.0,194.0,209.0,144.0,6.0
park-cemetery-etc-Bronx,8.0,9.0,8.0,5.0,28.0,16.0,47.0,23.0,29.0,1.0
park-cemetery-etc-Brooklyn,NaN,1.0,2.0,1.0,1.0,4.0,1.0,NaN,NaN,NaN


In [94]:
df_analysis_pivot['d2012'] = (df_analysis_pivot[2012] - df_analysis_pivot[2011])
df_analysis_pivot['d2013'] = (df_analysis_pivot[2013] - df_analysis_pivot[2012])
df_analysis_pivot['d2014'] = (df_analysis_pivot[2014] - df_analysis_pivot[2013])
df_analysis_pivot['d2015'] = (df_analysis_pivot[2015] - df_analysis_pivot[2014])
df_analysis_pivot['d2016'] = (df_analysis_pivot[2016] - df_analysis_pivot[2015])
df_analysis_pivot['d2017'] = (df_analysis_pivot[2017] - df_analysis_pivot[2016])
df_analysis_pivot['d2018'] = (df_analysis_pivot[2018] - df_analysis_pivot[2017])

In [ ]:
df_analysis_pivot.to_csv('df_analysis_pivot.csv')